In [1]:
import os
import ee
import geemap
import geopandas as gpd
import pandas as pd

from datetime import datetime, timedelta
from operator import itemgetter
from collections import defaultdict

import sys
sys.path.insert(0, "../src")

# change
from etl import *
from viz import *

In [2]:
os.chdir("..")

if not os.path.exists("images/"):
    os.mkdir("images/")

In [3]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWh5rbhJW-XaI_fjz5RPf10gTyJ1iQ8Wd0Y6Zw05CvB82fofvk3zMzY

Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
# simplified NLCD land cover classes
groupedLC = {1: "other",       # open water / perennial ice+snow
             2: "developed",
             3: "other",       # barren land
             4: "forest",
             5: "shrub",
             7: "grassland",
             8: "agriculture",
             9: "other"}       # wetland


def groupByLandCover(x, fireName):
    ddict = defaultdict(float)
    ddict["agriculture"] = 0   # regions without agriculture
    
    for key, value in x.items():
        newKey = int(key[0])
        ddict[groupedLC[newKey]] += value
        
    sumPixels = sum(ddict.values())
    for key, value in ddict.items():
        ddict[key] = 100*np.round(value/sumPixels, 3)
        
    ddict = dict(sorted(ddict.items()))
    return [fireName]+list(ddict.values())

In [6]:
nlcd = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL'
        ).filter(ee.Filter.eq('system:index', '2016')
        ).first()

## Aggregated NLCD Stats

In [7]:
bounds_EE = geemap.shp_to_ee("data/bounds/bounds.shp")

In [8]:
landCover = nlcd.select("landcover"
               ).reduceRegions(collection=bounds_EE,
                               reducer=ee.Reducer.frequencyHistogram(),
                               scale=30,
                               tileScale=4).getInfo()

results = []
for i in landCover["features"]:
    fireName = i["properties"]["FIRE_NAME"]
    results.append(groupByLandCover(i["properties"]["histogram"], fireName))
    
nlcd_1 = pd.DataFrame(results,
                    columns=["Fire", "Agriculture (%)", "Developed (%)",
                             "Forest (%)", "Grassland (%)",
                             "Other (%)", "Shrub (%)"])

nlcd_1

,Fire,Agriculture (%),Developed (%),Forest (%),Grassland (%),Other (%),Shrub (%)
0,ATLAS,10.6,12.0,26.1,17.5,2.8,30.9
1,BALD,0.4,0.7,34.4,36.9,1.0,26.6
2,BUCK,0.0,5.1,65.0,6.6,0.0,23.3
3,BUTTE,0.0,2.7,39.9,35.3,0.2,21.8
4,CALDWELL,8.7,1.4,18.7,11.5,2.3,57.4
5,CAMP,5.5,4.7,46.4,13.8,2.0,27.5
6,CARR,0.0,7.3,53.8,4.1,4.8,30.0
7,CASCADE,0.0,3.2,51.6,25.1,1.0,19.0
8,COVE,4.6,0.4,50.0,14.7,0.3,30.0
9,FRYING PAN,0.3,5.4,49.5,26.2,0.9,17.8


In [9]:
nlcdKeys = ["percent_tree_cover", "rangeland_annual_herbaceous",
            "rangeland_bare_ground", "rangeland_big_sagebrush", "rangeland_herbaceous",
            "rangeland_litter", "rangeland_sagebrush", "rangeland_shrub"]

nlcdStats = nlcd.select(nlcdKeys
               ).reduceRegions(collection=bounds_EE,
                               reducer=ee.Reducer.mean(),
                               scale=30,
                               tileScale=4).getInfo()

nlcdStats = list(map(lambda x: list(itemgetter(*(["FIRE_NAME"]+nlcdKeys))(x["properties"])),
                     nlcdStats["features"]))

# Slightly weird definitions + some missing values / probably not helpful
nlcd_2 = pd.DataFrame(nlcdStats, columns = ["Fire"]+nlcdKeys)
nlcd_2

,Fire,percent_tree_cover,rangeland_annual_herbaceous,rangeland_bare_ground,rangeland_big_sagebrush,rangeland_herbaceous,rangeland_litter,rangeland_sagebrush,rangeland_shrub
0,ATLAS,21.003382,78.415677,74.218033,70.728651,79.853776,75.715672,70.738710,79.386111
1,BALD,13.729520,34.643916,57.687153,33.915586,45.827852,46.887296,35.273469,39.933395
2,BUCK,52.313533,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BUTTE,42.768486,59.412316,57.920346,50.996851,63.631253,64.424698,50.996851,61.211486
4,CALDWELL,9.424805,41.338089,61.275747,35.806659,44.988651,44.166170,39.869653,42.837654
5,CAMP,36.020687,79.475732,77.549716,71.886171,82.783818,77.635572,71.886171,78.288198
6,CARR,43.627942,70.581317,72.870678,66.902461,73.195372,73.235247,66.905364,80.284333
7,CASCADE,34.354835,75.046037,61.692940,57.106382,77.442494,63.053891,57.106382,65.024445
8,COVE,27.057091,66.710335,69.114058,60.486779,72.056635,66.826855,61.422601,65.509498
9,FRYING PAN,54.730396,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Land Cover, L8, Burn Severity   Images/Gifs

In [10]:
burnPalette = ["706c1e", "4e9d5c", "fff70b", "ff641b", "a41fd6"]
landCoverPalette = ["A2D6F2", "FF7F68", "258914", "FFF100", "7CD860", "B99B56"]

# Params for saving images as png
landsatParams = {"dimensions": None,     # Landsat Bands 7-5-3
                 "region": None,
                 "min": 1000, "max": 25000,
                 "gamma":[1, 1, 1.2],
                 "format": "png"}

severityParams = {"dimensions": None,
                  "region": None,
                  "min": 0, "max": 4,
                  "palette": burnPalette,
                  "format": "png"}

nlcdParams = {"dimensions": None,
              "region": None,
              "min": 0, "max": 5,
              "palette": landCoverPalette,
              "format": "png"}

In [11]:
bounds_df = gpd.read_file("data/bounds/bounds.shp")
bbox_EE = geemap.shp_to_ee("data/unburned/bbox.shp")

In [12]:
# CA state + norCal counties for NLCD images
counties = gpd.read_file("data/CA_Counties/CA_Counties_TIGER2016.shp")
sfLowerBound = counties[counties["NAME"]=="San Francisco"]["geometry"].bounds["maxy"].values[0]
norCal = counties.bounds.apply(lambda x: x[3]>sfLowerBound, axis=1)
norCalCounties_EE =  geemap.gdf_to_ee(counties[norCal])

ca_bounds = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM1_NAME", "California"))

In [13]:
nlcdSimple = nlcd.select("landcover"
                ).expression(" (b('landcover') > 90) ? 0 "    # blue: other (wetland)
                             ":(b('landcover') > 80) ? 5 "    # brown: agriculture
                             ":(b('landcover') > 70) ? 4 "    # lightGreen: grassland/herbaceous
                             ":(b('landcover') > 50) ? 3 "    # yellow: shrub
                             ":(b('landcover') > 40) ? 2 "    # green: forest
                             ":(b('landcover') > 30) ? 0 "    # blue: other (barren land)
                             ":(b('landcover') > 20) ? 1 "    # red: developed/urban
                             ":0"                             # blue: other (water/perennial ice+snow)
                ).rename("landCover")


# Land cover images for CA and NorCal
saveImage(image=nlcdSimple.clipToCollection(norCalCounties_EE
                         ).select("landCover"),
          params=nlcdParams, 
          options=[800, norCalCounties_EE.geometry().dissolve()],
          path="images/CA_landCover",
          fileName="norCal.png")

saveImage(image=nlcdSimple.clipToCollection(ca_bounds
                         ).select("landCover"),
          params=nlcdParams,
          options=[800, ca_bounds.geometry()],
          path="images/CA_landCover",
          fileName="ca.png")

In [14]:
gifText = {"l8": [], "severity": [], "landCover": []}

for fireName, preFireDate, postFireDate, geometry in bounds_df[["FIRE_NAME", "pre-date",
                                                                "post-date", "geometry"]].values:    
    outline = ee.Image(
               ).byte(
               ).paint(featureCollection=bbox_EE.filter(ee.Filter.eq("FIRE_NAME", fireName)),
                       color='fff70b',
                       width=2.5
               ).visualize(palette='fff70b') # doesn't work properly for burn seerity
    
    geometry = ee.Geometry.Rectangle(list(geometry.bounds))

    # Get pre-post fire Landsat 8 images
    preFireImage_l8 = mosaicByDate(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
                                    ).filterBounds(geometry
                                    ).filterDate(preFireDate,
                                                 ee.Date(preFireDate).advance(1, "day")))

    postFireImage_l8 = mosaicByDate(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
                                     ).filterBounds(geometry
                                     ).filterDate(postFireDate,
                                                  ee.Date(postFireDate).advance(1, "day")))

    preFireImage_l8, postFireImage_l8 = ee.Image(preFireImage_l8.get(0)), ee.Image(postFireImage_l8.get(0))

    # Calculate NBR, dNBR, and burn severity
    preFireNBR = preFireImage_l8.normalizedDifference(['SR_B5', 'SR_B7'])
    postFireNBR = postFireImage_l8.normalizedDifference(['SR_B5', 'SR_B7'])
    dNBR = (preFireNBR.subtract(postFireNBR)).multiply(1000).rename("dNBR")

    burnSeverity = dNBR.expression(" (b('dNBR') > 425) ? 4 "    # purple: high severity
                                   ":(b('dNBR') > 225) ? 3 "    # orange: moderate severity
                                   ":(b('dNBR') > 100) ? 2 "    # yellow: low severity
                                   ":(b('dNBR') > -60) ? 1 "    # green: unburned/unchanged
                                   ":0"                         # brown: vegetation growth
                      ).rename("burnSeverity")
    
    gifText["l8"].append("{}  {}".format(fireName, preFireDate))
    gifText["l8"].append("{}  {}".format(fireName, postFireDate))        
    gifText["severity"].append("{}  {}".format(fireName, postFireDate))
    gifText["landCover"].append("{}".format(fireName))
    
    paramOptions = [600, geometry]
    saveImage(image=preFireImage_l8.select(["SR_B7", "SR_B5", "SR_B3"]),
              params=landsatParams,
              options=paramOptions,
              path="images/l8/raw",
              fileName="{}_1.png".format(fireName))
    
    saveImage(image=postFireImage_l8.select(["SR_B7", "SR_B5", "SR_B3"]),
              params=landsatParams,
              options=paramOptions,
              path="images/l8/raw",
              fileName="{}_2.png".format(fireName))
    
    saveImage(image=burnSeverity.select("burnSeverity").blend(outline.select("vis-red")),
              params=severityParams,
              options=paramOptions,
              path="images/severity/raw",
              fileName="{}.png".format(fireName))
              
    saveImage(image=nlcdSimple,
              params=nlcdParams,
              options=paramOptions,
              path="images/landCover/raw",
              fileName="{}.png".format(fireName))

In [15]:
stdImageSize("images/l8/raw", 600, "images/l8/border")
stdImageSize("images/severity/raw", 600, "images/severity/border")
stdImageSize("images/landCover/raw", 600, "images/landCover/border")

makeGif("images/l8/border", "images/gifs/l8.gif", gifText["l8"], 1200)
makeGif("images/severity/border", "images/gifs/severity.gif", gifText["severity"], 1500)
makeGif("images/landCover/border", "images/gifs/landCover.gif", gifText["landCover"], 1500, ("45%", "96%"))

## Drought Gif

In [16]:
startDate = datetime.fromisoformat("1990-08-01")

dateSeq = []
for i in range(32):
    dateSeq.append(startDate.isoformat()[:10])
    startDate += timedelta(days=365.25)
    
droughtParams = {"dimension": 500,
                 "min": -5.5, "max": 5.5,
                 "region": ca_bounds.geometry(),
                 "palette": ["bf363a", "df745e", "f4ae91", "fcdccb", "faf4f1",
                             "d2e5ef", "9dcae1", "5da2cb", "2f78b3"]}

In [17]:
drought = ee.ImageCollection('GRIDMET/DROUGHT').filterDate("1990-01-01", "2022-01-01")

for i in range(len(dateSeq)):
    image = drought.filterDate(dateSeq[i],
                               ee.Date(dateSeq[i]).advance(1, "week")
                  ).mean(
                  ).select("pdsi")
    
    saveImage(image.clip(ca_bounds.geometry()), droughtParams,
              [600, ca_bounds.geometry()],
              "images/drought", "{}.png".format(dateSeq[i][:4])) 
    
makeGif("images/drought", "images/gifs/drought.gif", [i[:4] for i in dateSeq], 500, ("10%", "95%"), 25)